In [1]:
import json
import os
import numpy as np
import pandas as pd
from collections import defaultdict
import time

test = pd.read_json('./data/test.json', typ='frame')
tag_song = pd.read_json('./data/tag_song_pair.json', typ='frame', orient= 'table')
tags = pd.read_json("./data/tag_song_pair.json", orient="table")["tag"]
with open("./data/tag_similar_matrix50_2.json") as f:
    tag_similar_matrix50_2 = np.array(json.load(f))
song_meta = pd.read_json("./data/song_meta.json", typ = "frame")
train = pd.read_json("./data/train.json", typ = "frame")


genre_gn_all = pd.read_json('./data/genre_gn_all.json', typ='series', encoding='utf-8')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
gen_list = {}
for i in range(254):
    gen_list[genre_gn_all.loc[i]['gnr_code']] = 0
tag_list = tag_song['tag'].to_list()

def give_tag_song(idx):
    id_ = test.loc[idx]['id'].astype(str)
    tags = test.loc[idx]['tags']
    songs = test.loc[idx]['songs']
    return id_, tags, songs

def take_date(idx):
    target_date = test.loc[idx]['updt_date']
    year = target_date[:4]
    month = target_date[5:7]
    day = target_date[8:10]
    date = year+month+day
    return int(date)

def valid_date(idx, id_):
    plylst_date = take_date(idx)
    song_date = song_meta[song_meta['id']==id_]['issue_date'].to_list()[0]
    if song_date < plylst_date: return True
    else: return False

def setTagNumLimit(list): return len(list)+10
def tagIsTen(list, limit): return len(list) == limit
def songIsHund(list): return len(list) == 100
def resultCase_tag_song(idx):
    print(" song!!")
    id_, tags, songs = give_tag_song(idx)
    tagNumLimit = setTagNumLimit(tags)
    gen_list_copy = {}
    for i in genre_gn_all['gnr_code'].to_list():
        gen_list_copy[i] = gen_list[i]

    song_count = {}
    # 곡 빈도수    
    for s in songs:
        song_count[s] = 0
    for t in tags:
        if t in tag_list:
            for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
                if s in songs:
                    song_count[s] = song_count[s] + 1   

    tag_count = [0 for _ in range(29160)]
    # 태그 빈도수
    for i in range(29160):
        for s in tag_song.loc[i]['song']:
            if s in songs:
                tag_count[i] +=1
        
    # tag selection
    new_tags = []
    tag_count_copy = tag_count
    for i in range(tagNumLimit):
        target = tag_count_copy.index(max(tag_count_copy))
        if tag_count_copy[target] == 0: break
        if tag_song.loc[target]["tag"] not in new_tags: new_tags.append(tag_song.loc[target]['tag'])
        tag_count_copy[target] = 0   
    del tag_count_copy 
    if tagIsTen(new_tags, tagNumLimit) is False: print("[ERROR] Need More Tags")
    
    # song selection
    new_songs = []
    new_song_dic = {}
    left_num = 150
    for t in new_tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s not in songs:
                if s not in new_song_dic:
                    new_song_dic[s] = 0
                else: new_song_dic[s] = new_song_dic[s]+1
    while len(new_songs)<150:
        if len(new_song_dic.keys()) <= 0: break
        target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
        if valid_date(idx, target) and target not in songs and target not in new_songs:
            new_songs.append(target)
            for g in song_meta[song_meta['id']==target]['song_gn_dtl_gnr_basket'].to_list()[0]:
                gen_list_copy[g] += 1
        del new_song_dic[target]
    if songIsHund(new_songs) is False:
        new_song_dic = {}
        for i in range(29160):
            for s in tag_song.loc[i]['song']:
                if s not in songs:
                    if s not in new_song_dic:
                        new_song_dic[s] = 0
                    else: new_song_dic[s] = new_song_dic[s]+1
        left_num = 150 - len(new_songs)
        for i in range(left_num):
            if len(new_song_dic.keys()) <= 0: break
            target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
            if valid_date(idx, target) and target not in songs and target not in new_songs:
                new_songs.append(target)
                for g in song_meta[song_meta['id']==target]['song_gn_dtl_gnr_basket'].to_list()[0]:
                    gen_list_copy[g] += 1
            del new_song_dic[target]
        if songIsHund(new_songs) is False: print("[ERROR] Need More Songs")

    new_songs2 = []
    while True:
        if len(new_songs2)>=100: break
        tar_gen = max(gen_list_copy.keys(), key=(lambda k: gen_list_copy[k]))
        for i in new_songs:
            for j in song_meta[song_meta['id']==i]['song_gn_dtl_gnr_basket'].to_list()[0]:
                if j==tar_gen:
                    new_songs2.append(i)
                    break
            if len(new_songs2)>=100: break
        del gen_list_copy[tar_gen]
    
    #print("<Playlist {}>".format(idx))
    #print("- Before Filtering")
    #print("-- [Tags #]: " + str(len(new_tags)) + ", [Songs #]: " + str(len(new_songs)))
    result_list = []
    result_list.append(id_)
    for t in new_tags:
        if t in tags: new_tags.remove(t)
    new_tags = new_tags[:10]
    result_list.append(new_tags)
    result_list.append(new_songs2)
    #print("- After Filtering")
    #print("-- [Tags #]: " + str(len(new_tags)) + ", [Songs #]: " + str(len(new_songs2)))
    return result_list

def tag2idx(string):
    for i in range(29160):
        if string == tags[i]:
            return i

def tags2indices(tag_list):
    result = []

    for tag in tag_list:
        result.append(tag2idx(tag))
    return result
    
def idx2tag(idx):
    return tags[idx]

def pick_song(tag_idx_list):
    frequency_dict = defaultdict(int)
    for tag_idx in tag_idx_list:
        for song_idx in tag_song.loc[tag_idx]["song"]:
            frequency_dict[song_idx] += 1
    result = sorted(frequency_dict.items(), key = lambda item : item[1], reverse = True)
    return result



def my_func(test_case_idx):
    print("no songs!!")
    tag_list = test.loc[test_case_idx]["tags"]  #테스트 케이스는 여기서 val을 test로 바꿔주기
    tag_indices = tags2indices(tag_list)

    except_tags = []
    no_tags_idx = 0
    max_idx_list = np.array([],dtype = np.int32)
    for idx in tag_indices:
        max_idx_list = np.append(max_idx_list, tag_similar_matrix50_2[idx].argsort()[-20:])
    max_idx_list = np.sort(max_idx_list)[::-1]
    max_tags = np.array([])
    for i in max_idx_list:
        max_tags = np.append(max_tags, idx2tag(i))
    
    max_tags_top_10 = []
    for tag in tag_list:
        max_tags_top_10.append(tag)
        
    count = len(max_tags_top_10)
    i = 0
    
    while count < 10:
        if max_tags[i] not in max_tags_top_10:
            max_tags_top_10.append(max_tags[i])
            count += 1
        i +=1
    pick_all_song = pick_song(max_idx_list)
    max_song_top_100 = []
    max_song_idx = 0
    while len(max_song_top_100) < 100:
        song_num = pick_all_song[max_song_idx][0]
        update = train.loc[test_case_idx]["updt_date"]
        update = int(update[0:4] + update[5:7] + update[8:10])
        if(song_meta.loc[song_num]["issue_date"] < update):
            max_song_top_100.append(song_num)
        max_song_idx += 1
    # 잘 나오는지 테스트
    #if no_tags_idx < 5: 
    #    print("테스트케이스 번호 : {}\n10개의 가장 유사한 태그 : {}\n 노래 100개 : {}\n\n".format(test_case_idx, 
    #                                                                       max_tags_top_10, max_song_top_100))
    #for song_idx in max_song_top_100:
    #    print("{}".format(qwe(song_idx)), end = " / ")
    #no_tags_idx +=1
    result = []
    result.append(test.loc[test_case_idx]["id"])
    result.append(max_song_top_100)
    result.append(max_tags_top_10)
    return result

start = time.time()
result = []

from_ = 0
to_ = 1

for i in range(from_, to_):
    if len(test.loc[i]["songs"]) != 0:
        case = resultCase_tag_song(i)
    elif len(test.loc[i]["songs"]) ==0 and len(test.loc[i]["tags"]) !=0:
        case = my_func(i)
    else:
        print("{} has only title".format(i))
        case = [test.loc[i]["id"], [], []]
    result.append(case)
    print("{} th id : {} length_songs : {} length_tags : {}".format(i, result[i][0], len(result[i][1]), len(result[i][2])))

print(time.time() - start)

pd_result = pd.DataFrame(result, columns = ["id", "songs", "tags"])
pd_result.to_json("./result/result{}_{}.json".format(from_, to_), orient = "records")


no song!!
[ERROR] Need More Songs
0 th id : 118598 length_songs : 10 length_tags : 100
1 has only title
1 th id : 131447 length_songs : 0 length_tags : 0
no song!!
[ERROR] Need More Songs
2 th id : 51464 length_songs : 10 length_tags : 100
no song!!
[ERROR] Need More Songs
3 th id : 45144 length_songs : 10 length_tags : 100
no song!!
[ERROR] Need More Songs
4 th id : 79929 length_songs : 10 length_tags : 100
no song!!
[ERROR] Need More Songs
5 th id : 138538 length_songs : 10 length_tags : 100
no song!!
[ERROR] Need More Songs
6 th id : 127575 length_songs : 10 length_tags : 100
no song!!
[ERROR] Need More Songs
7 th id : 115317 length_songs : 10 length_tags : 100
no songs!!
8 th id : 80810 length_songs : 100 length_tags : 10
9 has only title
9 th id : 142007 length_songs : 0 length_tags : 0
173.93690490722656


In [6]:
asd = pd.read_json("./result/result{}_{}.json".format(from_,to_), orient = "records")

In [4]:
asd = pd.read_json("./data/test.json")

In [5]:
len(asd)

10740

In [9]:

from_ = 0
to_ = 1

for i in range(from_, to_):
    if len(test.loc[i]["songs"]) != 0:
        case = resultCase_tag_song(i)
    elif len(test.loc[i]["songs"]) ==0 and len(test.loc[i]["tags"]) !=0:
        case = my_func(i)
    else:
        print("{} has only title".format(i))
        case = [test.loc[i]["id"], [], []]
    result.append(case)
    print("{} th id : {} length_songs : {} length_tags : {}".format(i, result[i][0], len(result[i][1]), len(result[i][2])))

print(time.time() - start)

pd_result = pd.DataFrame(result, columns = ["id", "songs", "tags"])
pd_result.to_json("./result/result{}_{}.json".format(from_, to_), orient = "records")


no song!!
[ERROR] Need More Songs
0 th id : 118598 length_songs : 10 length_tags : 100
4020.0685741901398
